# Comparison of Fastai's image-based classifier and L.R.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

In [65]:
preds_valid = np.load('../data/output/fastai_predictions.npy')
preds_valid.shape

(4389, 207)

Indices of the training and validation samples used in the fastai classifier.

In [9]:
train_ids = pd.read_csv('../data/output/fastai_train_ids.csv')
valid_ids = pd.read_csv('../data/output/fastai_valid_ids.csv')

In [66]:
vocab = pd.read_csv('../data/output/fastai_vocab.csv')
target_names = vocab.values.ravel()
target_names[:5]

array(['11-beta-hsd1_inhibitor', '5-alpha_reductase_inhibitor',
       'acat_inhibitor', 'acetylcholine_receptor_agonist',
       'acetylcholine_receptor_antagonist'], dtype=object)

In [67]:
preds_valid = pd.DataFrame(preds_valid, columns=target_names)
preds_valid.shape

(4389, 207)

Remove the column containing 'none'

In [68]:
target_columns = targets.columns[1:]

In [69]:
preds_valid_fastai = preds_valid[target_columns].values

In [37]:
train = pd.read_csv('../data/train_features.csv')
targets = pd.read_csv('../data/train_targets_scored.csv')

In [38]:
valid = train[train.sig_id.isin(valid_ids.fname)]
train = train[train.sig_id.isin(train_ids.fname)]

In [39]:
y_train = targets[targets.sig_id.isin(train.sig_id)]
y_valid = targets[targets.sig_id.isin(valid.sig_id)]

In [40]:
y_train = y_train.iloc[:, 1:].to_numpy()
y_valid = y_valid.iloc[:, 1:].to_numpy()

In [41]:
coltrans = ColumnTransformer([
    ('drop_id', 'drop', 'sig_id'),
    ('ordencode', OrdinalEncoder(), ['cp_type', 'cp_dose'])
], remainder='passthrough')

pipe = Pipeline([
    ('preprocessing', coltrans),
    ('clf', OneVsRestClassifier(LogisticRegression(C=0.001)))
    
])

In [42]:
pipe.fit(train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('drop_id', 'drop', 'sig_id'),
                                                 ('ordencode', OrdinalEncoder(),
                                                  ['cp_type', 'cp_dose'])])),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=0.001)))])

In [60]:
preds_lr = pipe.predict_proba(valid)

In [61]:
log_loss(y_valid.ravel(), preds_lr.ravel())

0.017327907702963316

In [74]:
log_loss(y_valid.ravel(), preds_valid_fastai.ravel())

/home/giovenko/miniconda3/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2240: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


inf